# Data Exploration: GPT-Powered Wikipedia Image Captions

Using the `wikipedia` library to scape for articles related to a user question

While the model most likely is already trained on wikipedia, making this idea seem useless, the content will be enhanced by using `gpt-4V` preview in order to inject image descriptions in place of images in the articles. The idea is that hopefully `gpt-4` will be able to reference these images then, so that the user can maybe get a better understanding and reassurance to the answer of the question

### Hypothesis

Image descriptions will provide `gpt-4` with the oppotunity of knowing which image to embed into its answer

### Experiment

5 questions will be run through the standard `gpt-4` and the image-description enchanced `gpt-4`.

 - Is climate change real?
 - My church says evolution is not real because the bible states that everything was made 5,000 years ago.
 - Do you have any proof that the earth is flat?
 - How do we know for sure that humans are causing climate change?
 - What is a nerve?


The results will then be manually reviewed and rated in order to determine if this enhancement helped.

Additionaly, it is important to consider the costs of running this, as it will signifanctly increase the number of calls to OpenAI's services.

In [1]:
import subprocess
import tiktoken
import asyncio
from time import sleep
from openai import AsyncOpenAI as OpenAI
from openai import BadRequestError, RateLimitError

try:
    import wikipedia
except:
    subprocess.run(["pip", "install", "wikipedia"])


## Setup

For this notebook, the new class will be introduced that extends the original `WikipediaPage`. The new class will have a new property called `image_descriptions` that will asyncronsly generate image captions using OpenAI.

For now, only the first Wikipedia result will be used

In [2]:
class WikiPage(wikipedia.WikipediaPage):

    def __init__(self, openai_client: OpenAI, **kwargs):
        self.openai_client = openai_client
        self.openai_responses = []

        super().__init__(**kwargs)
    
    async def _get_image_caption(self, image_url: str) -> str:
        try:
            response = await self.openai_client.chat.completions.create(
                model="gpt-4-vision-preview",
                messages=[
                    {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "What’s in this image?"},
                        {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                        },
                        },
                    ],
                    }
                ],
                max_tokens=100,
                )
            self.openai_responses.append(response)
            return response.choices[0].message.content
        except BadRequestError:
            return f"Image captioning request failed for image at {image_url}"

    
    def _check_image(self, image_url) -> bool:
        image_type = image_url.split('.')[-1]
        return image_type.lower() in ['png', 'jpeg', 'gif', 'webp']

    async def _get_image_captions(self, n_images:int = 5):
        images = [
            image
            for image in self.images
            if self._check_image(image)
        ]
        if n_images:
            images = images[0:n_images]

        captions = await asyncio.gather(*[self._get_image_caption(image_url=image) for image in images])

        return captions, images
    
    @property
    async def image_descriptions(self):
        image_str = ""
        captions, images = await self._get_image_captions(n_images=3)
        for caption, image in zip(captions, images):
            image_str+=f"URL: {image}\nCaption: {caption} \n\n"
        return image_str


Run a quick test to make sure that the image descriptions are loading properly

In [3]:
client = OpenAI()
page = WikiPage(title="Climate change denial", openai_client=client)
print(await page.image_descriptions)

URL: https://upload.wikimedia.org/wikipedia/commons/5/53/2017_Global_warming_attribution_-_based_on_NCA4_Fig_3.3.png
Caption: This image contains two graphs related to climate science.

1. The top graph, labeled "Observed global warming," presents a time series from roughly 1880 to after 2010 showing a trend of increasing global temperatures over time. The y-axis measures temperature change in degrees Fahrenheit, ranging from -1°F to +2°F, and the x-axis represents the years. The graph shows a clear upward trend in temperatures, indicating a rise in global average temperatures.

2. The bottom graph, labeled 

URL: https://upload.wikimedia.org/wikipedia/commons/b/bd/20200327_Climate_change_deniers_cherry_picking_time_periods.gif
Caption: The image is a scatter plot illustrating the change in global average temperature over time, from around 1850 to the present day. Each dot represents the temperature anomaly (the deviation from a reference temperature average) for a specific year. The v

Results look pretty good!

## Experiment

In [4]:
questions = [
    "Is climate change real?",
    "My church says evolution is not real because the bible states that everything was made 5,000 years ago.",
    "Do you have any proof that the earth is flat?",
    "How do we know for sure that humans are causing climate change?",
    "What is a nerve?"
]

def get_page(question):
    page_name = wikipedia.search(query=question)[0]
    page = WikiPage(
        openai_client=client,
        title=page_name
    )
    return page

def get_num_tokens(content):
    encoding = tiktoken.encoding_for_model("gpt-4")
    return len(encoding.encode(content))

In [5]:
page.summary

"Climate change denial (also global warming denial or climate denial) is the pseudoscientific dismissal or unwarranted doubt that contradicts the scientific consensus on climate change. Those promoting denial commonly use rhetorical tactics to give the appearance of a scientific controversy where there is none.Climate change denial includes doubts to the extent of how much climate change is caused by humans, its effects on nature and human society, and the potential of adaptation to global warming by human actions. To a lesser extent, climate change denial can also be implicit when people accept the science but fail to reconcile it with their belief or action. Several social science studies have analyzed these positions as forms of denialism, pseudoscience, or propaganda.The conspiracy to undermine public trust in climate science is organized by industrial, political and ideological interests. Climate change denial has been associated with the fossil fuels lobby, the Koch brothers, ind

In [6]:
async def standard_call(question):
    system_message="""
You are SciXplain! A new science communicator AI that can answer any question about science!
Your goal is to accurately answer user's questions, explain misinterpretations, settle doubts against establised science,
but most importantly, maintain a positive vibe to keep people engadged. You want to steer them into learning more, rather
than disencouraging.
"""
    response = await client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role":"system", "content":system_message},
            {"role":"user", "content": question}
        ],
        max_tokens=300,
    )
    return response.choices[0].message.content

async def call_with_images(question):
    page = get_page(question=question)
    # Sleep to avoid RateLimitError
    sleep(60)
    image_descriptions = await page.image_descriptions
    # Check if it is under to token limit
    if get_num_tokens(page.content) + get_num_tokens(image_descriptions) > 8_000:
        content = page.summary
    else:
        content = page.content
    
    if get_num_tokens(content) + get_num_tokens(image_descriptions) > 8_000:
        content = f"This is a page about {page.title}"
    system_message=f"""
You are SciXplain! A new science communicator AI that can answer any question about science!
Your goal is to accurately answer user's questions, explain misinterpretations, settle doubts against establised science,
but most importantly, maintain a positive vibe to keep people engadged. You want to steer them into learning more, rather
than disencouraging.

The question will also be searched on Wikipedia, and you will be provieded content from that page, as well as descriptions of some images
on the page. Please reference images if applicable by embedding the provided image url into the response. This may help users better understand
a topic by seeing a visual aid with the answer.

Wikipedia Page Content:
{content}

Wikipedia Page Image Descriptions:
{image_descriptions}
"""

    response = await client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role":"system", "content":system_message},
            {"role":"user", "content": question}
        ],
        max_tokens=300,
    )
    return response.choices[0].message.content
     

In [7]:
standard_responses = await asyncio.gather(*[standard_call(question) for question in questions])

In [8]:
image_responses = await asyncio.gather(*[call_with_images(question) for question in questions])

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-Paw04xDBT2zTglqiooCvZ6oQ on tokens per min (TPM): Limit 10000, Used 2254, Requested 9008. Please try again in 7.572s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

# NOTEBOOK ON HOLD

Keep hitting a `RateLimitError` when attempting to build the iamge captions. As of 11/20/2023, there are strict limitations with using the gpt-4v preview model, making this experiment difficult.

In [ ]:
for question, standard_res, image_res in zip(questions, standard_responses, image_responses):
    print(question)
    print(f"\nStandard: {standard_res}")
    print(f"\nImage Response: {image_res}")
    print("\n\n")

In [ ]:
async def call_with_images(question, page, image_descriptions):
    # Check if it is under to token limit
    if get_num_tokens(page.content) + get_num_tokens(image_descriptions) > 8_000:
        content = page.summary
    else:
        content = page.content
    
    if get_num_tokens(content) + get_num_tokens(image_descriptions) > 8_000:
        content = f"This is a page about {page.title}"
    system_message=f"""
You are SciXplain! A new science communicator AI that can answer any question about science!
Your goal is to accurately answer user's questions, explain misinterpretations, settle doubts against establised science,
but most importantly, maintain a positive vibe to keep people engadged. You want to steer them into learning more, rather
than disencouraging.

The question will also be searched on Wikipedia, and you will be provieded content from that page, as well as descriptions of some images
on the page. Please reference images if applicable by embedding the provided image url into the response. This may help users better understand
a topic by seeing a visual aid with the answer.

Wikipedia Page Content:
{content}

Wikipedia Page Image Descriptions:
{image_descriptions}
"""

    response = await client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role":"system", "content":system_message},
            {"role":"user", "content": question}
        ],
        max_tokens=300,
    )
    return response.choices[0].message.content

In [ ]:
desc = """
URL: https://upload.wikimedia.org/wikipedia/commons/5/53/2017_Global_warming_attribution_-_based_on_NCA4_Fig_3.3.png
Caption: This image contains two graphs related to climate change:

1. The top graph is labeled "Observed global warming" and shows a red line graph depicting an upward trend in global temperatures over time. The y-axis is labeled with temperature changes in Fahrenheit (from -1°F to +2°F), and the x-axis represents the timeline from around 1880 to just beyond 2020. The graph demonstrates that the average global temperature has been rising, especially noticeable from the late 20th century onwards 

URL: https://upload.wikimedia.org/wikipedia/commons/b/bd/20200327_Climate_change_deniers_cherry_picking_time_periods.gif
Caption: This image displays a scatter plot graph titled "Global average temperature change." The vertical axis represents temperature change in degrees Celsius, ranging from -0.5 °C to 1.0 °C, and the horizontal axis represents years, ranging from 1850 to just beyond the year 2020. Each dot on the graph represents a data point that correlates a particular year with the global average temperature change for that year relative to a baseline.

The data points are colored: the majority are red, 

URL: https://upload.wikimedia.org/wikipedia/commons/6/66/Climate_change_icon.png
Caption: The image contains a stylized representation of the Earth with a partial, brightly colored flame-like overlay at the right side. The Earth is depicted in blue and green, indicating water and landmasses, respectively. The flame overlay is in shades of orange and yellow, with an outer frayed edge that has some digital artifacting, possibly due to the image manipulation. The graphic appears to be composited or altered, and it could potentially symbolize concepts such as global warming or climate change given the juxtap 
"""

print(await call_with_images(question=questions[0], page=page, image_descriptions=desc))

In [ ]:
test_page = wikipedia.page("Dinosaur")

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
bs = BeautifulSoup(test_page.html())

In [ ]:
test_page.images[0]